# Week 07

Supervised Learning: Classification

## Setup

Run the following 2 cells to import all necessary libraries and helpers for this week's exercises

In [ ]:
!wget -q https://github.com/PSAM-5020-2025S-A/5020-utils/raw/main/src/data_utils.py

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, classification_report

from data_utils import StandardScaler
from data_utils import LinearRegression
from data_utils import SGDClassifier, RandomForestClassifier, SVC, LogisticRegression
from data_utils import display_confusion_matrix, regression_error
from data_utils import object_from_json_url

## Regression

Regression, or Regression Analysis, is a set of statistical processes for estimating the relationship between a dependent variable (sometimes called the 'outcome', 'response' or 'label') and one or more independent variables (called 'features', 'dimensions' or 'columns').

For example, let's say we have the following data about people's wages and years of experience:

<img src="./imgs/wages-exp.png" width="620px"/>

We could use regression to calculate how the values for wages are affected by years of experience in our dataset, and then create a function to more generally estimate the relation between wages and experience:

<img src="./imgs/wages-exp-fit.png" width="620px"/>

We could now estimate wages for values of years of experience that we didn't have measurements for.

This is an estimate, but the more points we use and the more features we have in our dataset the better the regression results will be.

## More Regression!

Let's do one more regression exercise using a different dataset.

This one is for wine quality.

In [ ]:
## 1. Load Dataset
WINE_FILE = "https://raw.githubusercontent.com/PSAM-5020-2025S-A/5020-utils/main/datasets/json/wines.json"

# Read into DataFrame
wines_data = object_from_json_url(WINE_FILE)
wines_df = pd.DataFrame.from_records(wines_data)

# Look at features: values, types, names, etc
wines_df

### Analyze the data

Specifically, steps $3$: normalize and look at the covariance matrix using `quality` as the independent variable of interest.

In [ ]:
# TODO: normalize and look at covariance matrix

## 3. Normalize
scaler = StandardScaler()

wines_scaled = scaler.fit_transform(wines_df)

# Since this is a new dataset, let's peek at its covariance matrix

wines_scaled.cov()["quality"]

### Plot

Looks like `alcohol`, `density`, `chlorides` and `volatility` are the $4$ features that mostly contribute to the quality.

Let's look at graphs of `quality` as a function of these $4$ features.

This could be two $3D$ graphs of pairs of variables, but four $2D$ graphs is probably easier to read.

In [ ]:
# Plot output feature as a function of input features
def plot_feats(data, in_feats, out_feat):
  for feat in in_feats:
    x = data[feat]
    y = data[out_feat]
    plt.plot(x, y, marker='o', linestyle='', alpha=0.3)
    plt.xlabel(feat)
    plt.ylabel(out_feat)
    plt.show()

In [ ]:
feat_list = ["alcohol", "density", "chlorides", "volatility"]
plot_feats(wines_scaled, feat_list, "quality")

### Regression

Use the method we used last week in the diamond dataset to create a model that predicts wine `quality` as a function of **ALL** of its other features.

Use all of our features to run regression.

In [ ]:
# TODO: Create a linear model and run regression

## 4. Separate the outcome variable and the independent variables
feats = wines_scaled.drop(columns=["quality"])

## 5. Create a LinearRegression object
model_lin = LinearRegression()

# Create a model that relates quality of wines to all other features
model_lin.fit(feats, wines_scaled["quality"])

## 6. Run the model on the training data
pred_train_scaled = model_lin.predict(feats)

# Un-normalize the data
pred_train = scaler.inverse_transform(pred_train_scaled).apply(round)

## 7. Measure error
regression_error(wines_df["quality"], pred_train)

### Plot Results

On average our predictions are within $0.75$ points of the real quality values.

Save the original `quality` values in a variable called `quality_original` and the `predicted quality` in another variable, called`quality_predicted`, and run the cell below to look at some plots of our predictions.

In [ ]:
def plot_preds(data, prediction, in_feats, out_feat):
  for feat in in_feats:
    x = data[feat]
    y = data[out_feat]
    y_p = prediction[out_feat]

    # Plot the original quality
    plt.plot(x, y, marker='o', linestyle='', alpha=0.3)
    plt.plot(x, y_p, color='r', marker='o', linestyle='', alpha=0.3)
    plt.xlabel(feat)
    plt.ylabel(out_feat)
    plt.show()

In [ ]:
feat_list = ["alcohol", "density", "chlorides", "volatility"]
plot_preds(wines_df, pred_train, feat_list, "quality")

### Interpretation

# 🤔

Hmm.... these could be better.

Our model wasn't able to capture the fact that the calculated `quality` value should be a discrete value and not a number with decimals.

This is because our `quality` category is not continuous, and instead can only have particular discrete values.

Instead of trying to calculate continuous values for `quality`, our model should really be trying to put the wines in the right `quality` category.

Let's use a different type of model for this task.

Instead of learning how to predict a continuous value from the independent variables, like this:

<img src="./imgs/wages-exp-fit.png" width="620px"/>

Our model should learn how to place data points into discrete groups, like this:

<img src="./imgs/wages-exp-classes.png" width="620px"/>

## Classification

This is what's called a *classification* problem, or task (sometimes also called _Logistic Regression_).

Instead of trying to model the behavior of a continuous value, like `price` or `temperature`, a classification model tries to predict the correct *label* for given input data.

The steps for training a classification model are the same:

1. Load dataset
2. Encode label features as numbers
3. Normalize the data
4. Separate the outcome variable and the feature variables
5. Create a model
6. Run model on input data and test data, and 
7. Measure error

Even though we are trying to predict labels for our data, we still have to encode all label/categorical features into numbers, whether they are input or output variables. This is because our models are always using some kind of math to compare values and make predictions, and they wouldn't really know how to do that with text or other non-numeric data.

### Random ? Forest ?

The particular classification model we will use is called a [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html).

The model gets its name from another type of model called a [Decision Tree](https://scikit-learn.org/stable/modules/tree.html). During training, Decision Trees learn to model our data using simple decision rules, in a process that is conceptually similar to the game [Twenty Questions](https://en.wikipedia.org/wiki/Twenty_questions).

It learns to separate our data into categories by *asking* a series of yes/no, if/else, questions using our features:

<img src="./imgs/decision-trees.jpg" width="720px"/>

A Random Forest Classifier is a model that combines a bunch of Tree models that were trained with randomly selected subsets of our features.

### Let's Model !

Steps are the same as before:

In [ ]:
feats

In [ ]:
## 4. Separate the outcome variable and the independent variables

# We're still using scaled feature variables

# But, now our quality variable will be unscaled,
# so it keeps its original values as labels 0 - 9

## 5. Create a RandomForestClassifier object
model_rf = RandomForestClassifier(random_state=1010)

# Create a model that classifies quality of wines based on many features
model_rf.fit(feats, wines_df["quality"])

## 6. Run the model on the training data
predicted_train = model_rf.predict(feats)

## 7. Measure error
regression_error(wines_df["quality"], predicted_train["quality"])

### Plot Results

Ohh, that's better. On average, the model misses the quality value by about $0.07$ points.

Let's take a look at some plots to confirm that the model captured the discrete-ness of our label:

In [ ]:
plot_preds(wines_df, predicted_train, feat_list, "quality")

# 🍷🎉

Much better !

### Test Dataset

One thing that we haven't been doing, but is an important step when training any kind of ML model, is to actually test our model on data that wasn't used in the training.

When we calculate the regression/prediction error on the same dataset that we use to create the model, it tells us how well the model learned from the data. When we calculate this error on a test dataset, one that wasn't used for fitting the model, it tells us how well the model can generalize its predictions to data it hasn't seen.

The regression/prediction error is an indication of how good our model is.

Calculating it using a separate test dataset will tell us whether our model has actually learned patterns from the data or just memorized the training data.

The steps for preparing the data and running it through the model are the same, except we don't have to `fit()` a scaler, nor a model. We'll just use the scaler and model that we trained with the `train` set to `transform()`/`predict()` the `test` data.

In [ ]:
## 1. Load Dataset
WINE_TEST_FILE = "https://raw.githubusercontent.com/PSAM-5020-2025S-A/5020-utils/main/datasets/json/wines-test.json"

# Read into DataFrame
wines_test_data = object_from_json_url(WINE_TEST_FILE)
wines_test_df = pd.DataFrame.from_records(wines_test_data)

## 3. Normalize
wines_test_scaled = scaler.transform(wines_test_df)

## 4. Separate the independent variables
features_test = wines_test_scaled.drop(columns=["quality"])

## 6. Run the model on the test data
predicted_test = model_rf.predict(features_test)

## 7. Measure error
regression_error(wines_test_df["quality"], predicted_test["quality"])

### Interpretation

This is not great. On average our model is within $0.7$ quality points of the real values.

Let's take a look at some plots:

In [ ]:
plot_preds(wines_test_df, predicted_test, feat_list, "quality")

### Better Interpretation

Using the `regression_error()` function like this to measure the error of our classifier model gives us some sense of what's happening in the model, but we have better ways of analyzing the results of a classification task.

### Confusion Matrix

The **_Confusion Matrix_** graph is an example of an evaluation metric that is specifically used for measuring how well a classification model performs.

It's a very elegant way of displaying which classes our model can classify, which classes it confuses with other classes, and if there are any classes that it prefers to predict. There's actually a lot of information in that graph, that can tell us different things about our model and its potential shortcomings.

Let's look at a simpler **_Confusion Matrix_** and break down all the information that it displays:

<img src="./imgs/confusion2.jpg" height="350px" />

The above matrix for a fictitious model shows that we are interested in classifying things into two categories, with labels $0$ and $1$. Our data set contains $7$ instances of the $0$ class (sum of first row), and $10$ instances of class $1$ (sum of second row).

Our model correctly predicted $5$ of the $7$ instances of class $0$; while correctly predicting $7$ of the $10$ instances of class $1$. These are called the `True` classifications.

#### Accuracy

We can use these numbers to compute the **_Accuracy_** of our model, or, how often it gets the correct answer:

$\displaystyle Accuracy = \frac{T_0 + T_1}{Total\ Instances}$

And for our example model:

$\displaystyle Accuracy = \frac{5 + 7}{5+2+3+7} = \frac{12}{17} \approx 0.7059 = 70.6\%$

Besides the **_Accuracy_** of our model, we can also compute **_Precision_** and **_Recall_** characteristics for each of the classes in our model.

#### Precision

**_Precision_** is the proportion of our predictions that were actually correct for any given class. This number will be low if our model is biased to predict any of the classes.

$\displaystyle Precision_c = \frac{T_c}{T_c + F_c}$

$\displaystyle Precision_0 = \frac{5}{5 + 3} = 0.625 = 62.5\%$ $\hspace{5em}$
$\displaystyle Precision_1 = \frac{7}{7 + 2} \approx 0.7778 = 77.78\%$

#### Recall 

**_Recall_** is the proportion of the true data labels that were predicted correctly for each class. This number will be low if our model can't really recognize one of the classes.

$\displaystyle Recall_c = \frac{T_c}{Total_c}$

$\displaystyle Recall_0 = \frac{5}{5 + 2} \approx 0.7143 = 71.43\%$ $\hspace{5em}$
$\displaystyle Recall_1 = \frac{7}{7 + 3} = 0.7 = 70\%$

### Many Classes

The exact same calculations can be done on any confusion matrix, independent of the number of classes in our model.

Consider the following matrix for a classification model with $3$ classes:

<img src="./imgs/confusion3.jpg" height="350px" />

#### Accuracy

This is still $\frac{correct}{total}$. So, for our model above:

$\displaystyle Accuracy = \frac{5 + 7 + 6}{5+2+1+2+7+2+3+0+6} = \frac{18}{28} \approx 0.6428 = 64.3\%$

#### Precision

The proportion of our predictions that were actually correct for any given class.

$\displaystyle Precision_0 = \frac{5}{5 + 2 + 3} = 0.5 = 50\%$ $\hspace{5em}$
$\displaystyle Precision_1 = \frac{7}{7 + 2 + 0} \approx 0.7778 = 77.78\%$

$\displaystyle Precision_2 = \frac{6}{6 + 2 + 1} \approx 0.6667 = 66.67\%$

#### Recall

The proportion of the true data labels that were predicted correctly for each class.

$\displaystyle Recall_0 = \frac{5}{5 + 2 + 1} = 0.625 = 62.5\%$ $\hspace{5em}$
$\displaystyle Recall_1 = \frac{7}{7 + 2 + 2} \approx 0.6364 = 63.64\%$

$\displaystyle Recall_2 = \frac{6}{6 + 0 + 3} \approx 0.6667 = 66.67\%$

### Evaluations

Ideally we would have $100\%$ for all of these metrics, but that's rarely the case.

They become useful when we have to consider tradeoffs between having models that are too sensitive and eager to predict certain classes, which might lead to low **_precision_**, versus models that don't predict certain classes ever, which would lead to low **_recall_**.

For example, if we are working on a face recognition system that will be used to unlock people's devices, we can probably live with lower **_recall_**, as long as our **_precision_** is high. This would mean that the system wouldn't always recognize certain faces easily, making it hard for some people to unlock their devices, but, on the other hand, the high **_precision_** would mean that it would be rare for the system to unlock the device for the wrong face.

### Wine Confusion

Let's look at the confusion matrices for our wine model.

First, the training data:

In [ ]:
display_labels = [q for q in range(wines_df["quality"].min(), wines_df["quality"].max()+1)]

display_confusion_matrix(wines_df["quality"], predicted_train["quality"], display_labels)

Now, the test data:

In [ ]:
display_confusion_matrix(wines_test_df["quality"], predicted_test["quality"], display_labels)

### Calculating Accuracy, Precision, Recall

Let's... use the `Scikit-Learn` metric functions to evaluate our model.

We can use `accuracy_score()`, `precision_score()` and `recall_score()` by just passing our $2$ lists of true labels and predicted labels.

For calculating multi-class `precision_score()` and `recall_score()` we also have to specify how the class precisions should be combined into a final score, if at all.

If we want to see per-class precision and recall, we just have to pass `average=None` as a parameter to `precision_score()` and `recall_score()`.

Sometimes we also have to tell the function what to do to avoid a division-by-zero when a row/column has no predictions. We do that by parring `zero_division=0` and telling it to assign a precision/recall score of $0$ whenever a row/column has no predictions.

In [ ]:
print(classification_report(wines_df["quality"].values, predicted_train["quality"].values))

In [ ]:
def print_classification_scores(labels, predictions):
  print("Accuracy:", accuracy_score(labels, predictions))
  print("Precision:", precision_score(labels, predictions, average=None, zero_division=0).round(3))
  print("Recall:", recall_score(labels, predictions, average=None, zero_division=0).round(3))

In [ ]:
print("Train")
print_classification_scores(wines_df["quality"].values, predicted_train["quality"].values)

print("Test")
print_classification_scores(wines_test_df["quality"].values, predicted_test["quality"].values)

In [ ]:
print("Train")
print(classification_report(wines_df["quality"].values, predicted_train["quality"].values, zero_division=0))

print("Test")
print(classification_report(wines_test_df["quality"].values, predicted_test["quality"].values, zero_division=0))

### Interpretation

We have an *ok* model for predicting wine quality based on a few parameters.

The scatter plots and the confusion matrix for the test dataset show that our model doesn't perform very well for the $2$ lowest and $2$ highest quality groups. This can be because of the scarcity of data points for those groups, as it never learns to even predict $3$ or $9$.

The confusion matrix also shows that the model has a hard time with group $7$, where it predicts group $6$ about $40\%$ of the time.

### Iterate

Repeat the classification, but using one of the other modeling techniques: `SGDClassifier`, `SVC` or `LogisticRegression`.

The data is already scaled and in separate `DataFrame`s. We just have to:
- Create a Classifier object
- Run `fit()` on the training data
- Evaluate using the training and test datasets

Does it make sense to pick one over the others ? Is it easier, harder or the same to set up and pick parameters ? Are the results very different ?

In [ ]:
# TODO: re-fit the data using a different modeling technique

## 5. Create a new Classifier object
model_log = LogisticRegression(max_iter=1000, random_state=1010)
model_log.fit(feats, wines_df["quality"])

## 6. Run the model on the training data
predicted_train = model_log.predict(feats)

## 7. Measure error
display(regression_error(wines_df["quality"], predicted_train))

## 7b. Look at graphs
plot_preds(wines_df, predicted_train, feat_list, "quality")

## 8. Look at Confusion Matrices
display_confusion_matrix(wines_df["quality"], predicted_train["quality"], display_labels)

## 8b. Print scores
print("Train")
print(classification_report(wines_df["quality"].values, predicted_train["quality"].values, zero_division=0))

In [ ]:
## 6. Run the model on the training data
predicted_test = model_log.predict(features_test)

## 7. Measure error
display(regression_error(wines_test_df["quality"], predicted_test))

## 7b. Look at graphs
plot_preds(wines_test_df, predicted_test, feat_list, "quality")

## 8. Look at Confusion Matrices
display_confusion_matrix(wines_test_df["quality"], predicted_test["quality"], display_labels)

## 8b. Print scores
print("Test")
print(classification_report(wines_test_df["quality"].values, predicted_test["quality"].values, zero_division=0))

### Class Probabilities

# TODO HERE

# then amend commit

In [ ]:
model_log.predict_proba(feats[:4].values)

### Conclusion

# TODO HERE
# then amend commit